In [1]:
import pandas as pd
import os

In [2]:
BASE_DIR = os.path.join('..', 'data')
RAW_DIR = os.path.join(BASE_DIR, 'raw')
ENGINEERED_DIR = os.path.join(BASE_DIR, 'engineered')

In [26]:
PLAYERS_DICT = [
        "advanced goalkeeping",
        "defensive",
        "goal and shot creation",
        "goalkeeping",
        "misc",
        "pass types",
        "passing",
        "playing time",
        "possession",
        "shooting",
        "standard",
]

In [27]:
output_dir = os.path.join(ENGINEERED_DIR, '2023')
input_dir_europe = os.path.join(RAW_DIR, '2024')
input_dir_south_america = os.path.join(RAW_DIR, '2023')
def process_folder(folder_path):
    folder_dict = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)

        folder_dict.append(df)

    return folder_dict

def process_folders(root_folder):
    folder_dicts = {}

    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)

        if os.path.isdir(folder_path):
            folder_dict = process_folder(folder_path)
            folder_dicts[folder_name] = folder_dict

    return folder_dicts

In [28]:
dicts_europe = process_folders(input_dir_europe)
dicts_europe.keys()

dict_keys(['2. Bundesliga', 'Big 5 combined', 'EFL Championship', 'Eredivisie', 'Jupiler Pro League', 'La Liga 2', 'Liga MX', 'Ligue 2', 'Primeira Liga', 'Serie B'])

In [29]:
dicts_sa = process_folders(input_dir_south_america)
dicts_sa.keys()

dict_keys(['Argentina Liga Profesional', 'Brazilian Serie A'])

In [40]:
big_5_combined = dicts_europe["Big 5 combined"]
bundesliga_2 = dicts_europe["2. Bundesliga"]
efl_championship = dicts_europe["EFL Championship"]
eredivisie = dicts_europe["Eredivisie"]
jupiler = dicts_europe["Jupiler Pro League"]
laliga_2 = dicts_europe["La Liga 2"]
liga_mx = dicts_europe["Liga MX"]
ligue_2 = dicts_europe["Ligue 2"]
primeira_liga = dicts_europe["Primeira Liga"]
serie_b = dicts_europe["Serie B"]
brazil_serie_a = dicts_sa["Brazilian Serie A"]
argentina = dicts_sa["Argentina Liga Profesional"]

In [41]:
final_df = {}
for i in range(len(big_5_combined)):
    to_concat = []
    to_concat.append(big_5_combined[i])
    to_concat.append(bundesliga_2[i])
    to_concat.append(efl_championship[i])
    to_concat.append(eredivisie[i])
    to_concat.append(jupiler[i])
    to_concat.append(laliga_2[i])
    to_concat.append(liga_mx[i])
    to_concat.append(ligue_2[i])
    to_concat.append(primeira_liga[i])
    to_concat.append(serie_b[i])
    to_concat.append(brazil_serie_a[i])
    to_concat.append(argentina[i])
    final_df[PLAYERS_DICT[i]] = pd.concat(to_concat, axis=0)



In [47]:
output_dir = os.path.join(ENGINEERED_DIR, '2023', 'players_stats')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

for key, val in final_df.items():
    file_path = os.path.join(output_dir, key + ".csv")
    val.to_csv(file_path, index=False)